## **Serie A 22/23 Predictions**

In [1]:
# imports
import pandas as pd
import numpy as np
from scipy.stats import poisson 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

problems

- Hellas Verona: saved as 'Verona' in serieAData and as 'Hellas Verona' in serieATable

Substitute the term 'Verona' in serieATable

- Cremonese & Monza: no data from previous seasons

Assign them to other teams or assign my average based on other low-level teams from past years

Cremonese : Venezia | Monza : Genoa

Data

In [2]:
# fetch dataset: Serie A data since season 2005-06
dfList = []
for i in range(5, 22):
    df = pd.read_csv("../data/historical-seriea/SerieA-{0:02d}{1:02d}-I1.csv".format(i,i+1))
    df = df[['HomeTeam','AwayTeam','FTHG','FTAG']]
    df = df.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})
    dfList.append(df)
    
serieAData = dfList[0].append(dfList[1:])

C:\Users\FC\AppData\Local\Temp\ipykernel_10200\1015701821.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  serieAData = dfList[0].append(dfList[1:])


In [3]:
serieAData.to_csv('../data/serieAData.csv', index=False)

In [4]:
# fetch current standings (as of 13-08-2022)
serieATable = pd.read_csv('../data/SerieA-2223-standings-OK.csv')[['Squad', 'Pts']]
serieATable.index = np.arange(1, len(serieATable) + 1)

serieATable.rename(columns={"Squad": "Team", "Pts": "Points"}, inplace=True)
serieATable['Team'] = serieATable['Team'].str.replace('Hellas Verona','Verona')

In [5]:
# fetch upcoming match fixures
serieAMatches = pd.read_csv('../data/SerieA-2223-calendar-OK.csv')

serieAMatches.rename(columns={"Home": "HomeTeam", "Away": "AwayTeam"}, inplace=True)
serieAMatches['HomeTeam'] = serieAMatches['HomeTeam'].str.replace('Hellas Verona','Verona')
serieAMatches['AwayTeam'] = serieAMatches['AwayTeam'].str.replace('Hellas Verona','Verona')

In [6]:
'PAST MATCH DATA: {} rows'.format(len(serieAData))
serieAData.head()
'CURRENT POINTS TABLE'
serieATable
'UPCOMING MATCH FIXTURES: {} rows'.format(len(serieAMatches))
serieAMatches.head()

'PAST MATCH DATA: 6465 rows'

,HomeTeam,AwayTeam,HomeGoals,AwayGoals
0,Fiorentina,Sampdoria,2.0,1.0
1,Livorno,Lecce,2.0,1.0
2,Ascoli,Milan,1.0,1.0
3,Inter,Treviso,3.0,0.0
4,Juventus,Chievo,1.0,0.0


'CURRENT POINTS TABLE'

,Team,Points
1,Juventus,0
2,Napoli,0
3,Inter,0
4,Fiorentina,0
5,Lazio,0
6,Roma,0
7,Milan,0
8,Udinese,0
9,Atalanta,0
10,Sampdoria,0


'UPCOMING MATCH FIXTURES: 380 rows'

,HomeTeam,AwayTeam
0,Milan,Udinese
1,Sampdoria,Atalanta
2,Lecce,Inter
3,Monza,Torino
4,Lazio,Bologna


Calculate Team and League stats

In [7]:
# for each team - average {HomeScored, HomeConceded, AwayScored, AwayConceded} 
serieAHomeTeam = serieAData[['HomeTeam', 'HomeGoals', 'AwayGoals']].rename(
    columns={'HomeTeam':'Team', 'HomeGoals':'HomeScored', 'AwayGoals':'HomeConceded'}).groupby(
    ['Team'], as_index=False)[['HomeScored', 'HomeConceded']].mean()

serieAAwayTeam = serieAData[['AwayTeam', 'HomeGoals', 'AwayGoals']].rename(
    columns={'AwayTeam':'Team', 'HomeGoals':'AwayConceded', 'AwayGoals':'AwayScored'}).groupby(
    ['Team'], as_index=False)[['AwayScored', 'AwayConceded']].mean()

In [8]:
# overall - Average {leagueHomeScored, leagueHomeConceded, leagueAwayScored, leagueAwayConceded}
leagueHomeScored, leagueHomeConceded = serieAHomeTeam['HomeScored'].mean(), serieAHomeTeam['HomeConceded'].mean()
leagueAwayScored, leagueAwayConceded = serieAAwayTeam['AwayScored'].mean(), serieAAwayTeam['AwayConceded'].mean()

In [9]:
serieATeamStrength = pd.merge(serieAHomeTeam, serieAAwayTeam, on='Team')

assert(leagueHomeScored != 0)
assert(leagueHomeConceded != 0)
assert(leagueAwayScored != 0)
assert(leagueAwayConceded != 0)

In [10]:
# normalize the parameters 
# for each team - {HomeAttack, HomeDefence, AwayAttack, AwayDefense}
serieATeamStrength['HomeScored'] /= leagueHomeScored
serieATeamStrength['HomeConceded'] /= leagueHomeConceded
serieATeamStrength['AwayScored'] /= leagueAwayScored
serieATeamStrength['AwayConceded'] /= leagueAwayConceded

serieATeamStrength.columns=['Team','HomeAttack','HomeDefense','AwayAttack','AwayDefense']
serieATeamStrength.set_index('Team', inplace=True)

# overall - {overallHomeScored, overallAwayScored}
overallHomeScored = (leagueHomeScored+leagueAwayConceded)/2
overallAwayScored = (leagueHomeConceded+leagueAwayScored)/2

In [11]:
# cremonese : venezia
# monza : genoa

serieATeamStrength.reset_index(inplace=True)

In [12]:
new_row = {'Team':'Monza', 'HomeAttack':1.082263, 'HomeDefense':0.950955, 'AwayAttack':0.900825, 'AwayDefense':0.946123}
new_row2 = {'Team':'Cremonese', 'HomeAttack':0.837623, 'HomeDefense':1.485054, 'AwayAttack':0.670461, 'AwayDefense':0.966916}

#append row to the dataframe
serieATeamStrength = serieATeamStrength.append(new_row, ignore_index=True)
serieATeamStrength = serieATeamStrength.append(new_row2, ignore_index=True)

#serieATeamStrength.loc[len(serieATeamStrength)] = ['Monza', 1.082263, 0.950955, 0.900825, 0.946123]
#serieATeamStrength.loc[len(serieATeamStrength)] = ['Cremonese', 0.837623, 1.485054, 0.670461, 0.966916]

serieATeamStrength.set_index('Team', inplace=True)

C:\Users\FC\AppData\Local\Temp\ipykernel_10200\3468571818.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  serieATeamStrength = serieATeamStrength.append(new_row, ignore_index=True)
C:\Users\FC\AppData\Local\Temp\ipykernel_10200\3468571818.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  serieATeamStrength = serieATeamStrength.append(new_row2, ignore_index=True)


In [13]:
'TEAM STRENGTH: {} rows'.format(len(serieATeamStrength))
serieATeamStrength
'Overall Home scored = {}'.format(overallHomeScored)
'Overall Away scored = {}'.format(overallAwayScored)

'TEAM STRENGTH: 43 rows'

,HomeAttack,HomeDefense,AwayAttack,AwayDefense
Team,,,,
Ascoli,1.017114,1.113790,0.722035,0.982512
Atalanta,1.247128,0.875400,1.279034,0.827597
Bari,0.837623,0.840228,0.876757,0.966916
Benevento,0.837623,1.543674,0.799396,1.247634
Bologna,0.972625,1.001058,0.896593,0.923729
Brescia,0.857567,1.133330,0.670461,1.138466
Cagliari,0.997171,0.977009,0.905767,0.998107
Carpi,0.917397,1.016089,0.722035,0.966916
Catania,1.027086,0.752297,0.831630,1.056590


'Overall Home scored = 1.503463569171396'

'Overall Away scored = 1.1836300276594205'

In [14]:
# predict outcome of match and assign points to the teams

def predictMatchScore(home, away):
    if home in serieATeamStrength.index and away in serieATeamStrength.index:
        lambdH = serieATeamStrength.at[home,'HomeAttack'] * serieATeamStrength.at[away,'AwayDefense'] * overallHomeScored
        lambdA = serieATeamStrength.at[away,'AwayAttack'] * serieATeamStrength.at[home,'HomeDefense'] * overallAwayScored
        probH, probA, probT = 0, 0, 0  # probability of Home win(H), Away win(A) or Tie(T) 
        for X in range(0,11):
            for Y in range(0, 11):
                p = poisson.pmf(X, lambdH) * poisson.pmf(Y, lambdA)
                if X == Y:
                    probT += p
                elif X > Y:
                    probH += p
                else:
                    probA += p
        scoreH = 3 * probH + probT
        scoreA = 3 * probA + probT
        return (scoreH, scoreA)
    else:
        return (0, 0)

In [15]:
#  simulate the matches to predict final standings



for index, row in serieAMatches.iterrows():
    home, away = row['HomeTeam'], row['AwayTeam']
    assert(home in serieATable.Team.values and away in serieATable.Team.values)
    sH, sA = predictMatchScore(home, away)
    serieATable.loc[serieATable.Team == home, 'Points'] += sH
    serieATable.loc[serieATable.Team == away, 'Points'] += sA

In [16]:
'PREDICTED FINAL STANDINGS'
serieATable = serieATable.sort_values('Points', ascending=False)
serieATable.index = np.arange(1, len(serieATable)+1) 
serieATable.round(2)

'PREDICTED FINAL STANDINGS'

,Team,Points
1,Juventus,79.95
2,Inter,73.86
3,Napoli,72.31
4,Milan,70.88
5,Roma,70.48
6,Lazio,62.50
7,Fiorentina,60.94
8,Atalanta,58.40
9,Udinese,50.34
10,Sampdoria,50.22


In [19]:
serieATable = serieATable.round(2)
serieATable.to_csv("../data/serieATable-final.csv")